[View in Colaboratory](https://colab.research.google.com/github/jaisimhamanipatruni/Jai-/blob/master/Google_Dopamine.ipynb)

# **Motivations**
![alt text](https://camo.githubusercontent.com/5cbcb2c2c4d62d27517df605596d4b159163dc3d/68747470733a2f2f7777772e736b796e6574746f6461792e636f6d2f636f6e74656e742f656469746f7269616c732f696d616765732f69732d616c706861676f2d7a65726f2d6f76657272617465642f686973746f72792e706e67)



   Deep Reinforcment learning is responsible for the 2 biggest AI victories most recently (The Deep Q Network, AlphaGo, OpenAI Five)
Deep Q started the surge of interest in Deep RL
Deep Q was so impressive that Google bought DeepMind for half a billion US dollars to leverage it for their products
Deep Q exceled because of 3 (at the time) state-of-the art techniques; replay memory, large-scale distributed training, and distributional modeling methods

The time and attention it took Google researchers to develop these techniques made them realize they needed a framework that fit some key pain points


**Wait, Jaisimha, What's replay memory?**

To perform experience replay, an agent will store it's experiences t=(s,a,r,st+1)
So instead of running Q-learning on state/action pairs as they occur during simulation or actual experience, the system stores the data discovered for [state, action, reward, next_state] - typically in a large table. Note this does not store associated values - this is the raw data to feed into action-value calculations later.
The learning phase is then logically separate from gaining experience, and based on taking random samples from this table.
You still want to interleave the two processes - acting and learning - because improving the policy will lead to different behaviour that should explore actions closer to optimal ones, and you want to learn from those.



# Framework Features


# Agents

**DQN**
![alt text](https://camo.githubusercontent.com/bbf37a6e1873bda59a40400c34d8ffb7bdf93808/687474703a2f2f736877732e63632e6f6974612d752e61632e6a702f736869626174612f4b69737353797374656d2e706e67)

C51
Basically it is a good old Deep-Q network. But instead of approximating expected future reward values it generates the whole distributions over possible outcomes. -They’ve done it by replacing the output of DQN with probabilities over categorical variables representing different reward values ranges

# Markov Decision Processes
![alt text](https://camo.githubusercontent.com/563ddd4662659b2249a24cf85c5f323cb902d514/68747470733a2f2f696d6167652e736c696465736861726563646e2e636f6d2f6d6c2d7365702d30392d3039313030393134313631352d70687061707030312f39352f72656772657462617365642d7265776172642d656c696369746174696f6e2d666f722d6d61726b6f762d6465636973696f6e2d70726f6365737365732d33392d3732382e6a70673f63623d31323535303938313539)



![alt text](https://camo.githubusercontent.com/e430d0e45ea073550ed2c8f3f9528c3ee4651b46/68747470733a2f2f692e737461636b2e696d6775722e636f6d2f514e56375a2e6a7067)

# Policies

 ![alt text](https://camo.githubusercontent.com/f952dbd309d626a9abdba478bd080708c0010f18/68747470733a2f2f696d6167652e736c696465736861726563646e2e636f6d2f646565707265696e666f7263656d656e746c6561726e696e6766726f6d73637261746368312d3137313232313135333631342f39352f646565702d7265696e666f7263656d656e742d6c6561726e696e672d66726f6d2d736372617463682d32392d3633382e6a70673f63623d31353133383730363237)

![alt text](https://camo.githubusercontent.com/bbba90395278d0bdfadcf71b75a01935bb392777/68747470733a2f2f696d616765732e736c696465706c617965722e636f6d2f31362f343935383332382f736c696465732f736c6964655f31382e6a7067)

In [0]:
# @title Install necessary packages.
#dopamine for RL
!pip install --upgrade --no-cache-dir dopamine-rl
# dopamine dependencies
!pip install cmake
#Arcade Learning Environment
!pip install atari_py

In [0]:
# @title Necessary imports and globals.
#matrix math
import numpy as np
#load files
import os
#DQN for baselines
from dopamine.agents.dqn import dqn_agent
#high level agent-environment excecution engine
from dopamine.atari import run_experiment
#visualization + data downloading
from dopamine.colab import utils as colab_utils
#warnings
from absl import flags

#where to store training logs
BASE_PATH = '/tmp/colab_dope_run'  # @param
#which arcade environment?
GAME = 'Asterix'  # @param

In [0]:
# @title Create a  new agent from scratch.

#define where to store log data
LOG_PATH = os.path.join(BASE_PATH, 'basic_agent', GAME)

class BasicAgent(object):
  """This agent randomly selects an action and sticks to it. It will change
  actions with probability switch_prob."""
  def __init__(self, sess, num_actions, switch_prob=0.1):
    #tensorflow session
    self._sess = sess
    #how many possible actions can it take?
    self._num_actions = num_actions
    # probability of switching actions in the next timestep?
    self._switch_prob = switch_prob
    #initialize the action to take (randomly)
    self._last_action = np.random.randint(num_actions)
    #not debugging
    self.eval_mode = False
  
  #How select an action? 
  #we define our policy here
  def _choose_action(self):
    if np.random.random() <= self._switch_prob:
      self._last_action = np.random.randint(self._num_actions)
    return self._last_action
    
  #when it checkpoints during training, anything we should do?
  def bundle_and_checkpoint(self, unused_checkpoint_dir, unused_iteration):
    pass
    
  #loading from checkpoint
  def unbundle(self, unused_checkpoint_dir, unused_checkpoint_version,
               unused_data):
    pass
  
  #first action to take
  def begin_episode(self, unused_observation):
    return self._choose_action()
  
  #cleanup
  def end_episode(self, unused_reward):
    pass
  
  #we can update our policy here
  #using the reward and observation
  #dynamic programming, Q learning, monte carlo methods, etc.
  def step(self, reward, observation):
    return self._choose_action()
  
def create_basic_agent(sess, environment):
  """The Runner class will expect a function of this type to create an agent."""
  return BasicAgent(sess, num_actions=environment.action_space.n,
                     switch_prob=0.2)

# Create the runner class with this agent. We use very small numbers of steps
# to terminate quickly, as this is mostly meant for demonstrating how one can
# use the framework. We also explicitly terminate after 110 iterations (instead
# of the standard 200) to demonstrate the plotting of partial runs.
basic_runner = run_experiment.Runner(LOG_PATH,
                                      create_basic_agent,
                                      game_name=GAME,
                                      num_iterations=200,
                                      training_steps=10,
                                      evaluation_steps=10,
                                      max_steps_per_episode=100

In [0]:
 @title Train Basic Agent.
print('Will train basic agent, please be patient, may be a while...')
basic_runner.run_experiment()
print('Done training!')

In [0]:
# @title Load baseline data
!gsutil -q -m cp -R gs://download-dopamine-rl/preprocessed-benchmarks/* /content/
experimental_data = colab_utils.load_baselines('/content')

In [0]:
# @title Load the training logs.
basic_data = colab_utils.read_experiment(log_path=LOG_PATH, verbose=True)
basic_data['agent'] = 'BasicAgent'
basic_data['run_number'] = 1
experimental_data[GAME] = experimental_data[GAME].merge(basic_data,
                                                        how='outer')

In [0]:
# @title Plot training results.

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16,8))
sns.tsplot(data=experimental_data[GAME], time='iteration', unit='run_number',
           condition='agent', value='train_episode_returns', ax=ax)
plt.title(GAME)
plt.show()

![alt text](...%0A)